In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar los datos
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Procesar la columna 'Weight' y convertirla a numérico
train_df['Weight'] = train_df['Weight'].str.replace('kg', '').astype(float)
test_df['Weight'] = test_df['Weight'].str.replace('kg', '').astype(float)

# Procesar la columna 'Ram' y convertirla a numérico
train_df['Ram'] = train_df['Ram'].str.replace('GB', '').astype(int)
test_df['Ram'] = test_df['Ram'].str.replace('GB', '').astype(int)

# Extraer la resolución de pantalla
train_df[['res_width', 'res_height']] = train_df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(float)
test_df[['res_width', 'res_height']] = test_df['ScreenResolution'].str.extract(r'(\d{3,4})x(\d{3,4})').astype(float)

# Procesar la columna 'Memory' y convertirla a numérico
def parse_memory(memory_str):
    if isinstance(memory_str, str):
        memory_str = memory_str.strip().lower()
        if 'gb' in memory_str:
            return int(float(memory_str.split('gb')[0].strip()))
        elif 'tb' in memory_str:
            return int(float(memory_str.split('tb')[0].strip()) * 1024)
    return 0

train_df['Memory'] = train_df['Memory'].apply(parse_memory)
test_df['Memory'] = test_df['Memory'].apply(parse_memory)

# Seleccionar las características que usaremos para el modelo
features_to_use_improved = ['Inches', 'Ram', 'Weight', 'Memory', 'res_width', 'res_height']

# Crear los conjuntos de entrenamiento y prueba
X_train_improved = train_df[features_to_use_improved]
X_test_improved = test_df[features_to_use_improved]

# Separar la variable objetivo (target)
y_train = train_df['Price_euros']

# Dividir los datos en conjunto de entrenamiento y validación
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_improved, y_train, test_size=0.2, random_state=42)

# Aplicar la transformación logarítmica para hacer la distribución más normal
y_train_log = np.log1p(y_train_split)
y_val_log = np.log1p(y_val_split)


In [5]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

# Definir el modelo XGBoost
xgb_model = XGBRegressor(random_state=42, n_jobs=-1)

# Definir el espacio de búsqueda de los hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],  # Número de árboles
    'max_depth': [6, 10, 12],          # Profundidad máxima de los árboles
    'learning_rate': [0.01, 0.05, 0.1], # Tasa de aprendizaje
    'subsample': [0.7, 0.8, 1],         # Proporción de muestras a usar
    'colsample_bytree': [0.7, 0.8, 1],   # Proporción de características a usar
    'gamma': [0, 0.1, 0.2],            # Regularización gamma
    'alpha': [0, 0.1, 0.2],            # Regularización L1
    'lambda': [0, 0.1, 0.2]            # Regularización L2
}

# Crear el GridSearchCV con validación cruzada
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_absolute_error', 
                           n_jobs=-1, verbose=2)

# Ajustar el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train_split, y_train_log)

# Mostrar los mejores parámetros encontrados
best_params = grid_search.best_params_
print(f'Mejores parámetros encontrados: {best_params}')

# Predecir con el mejor modelo
best_xgb_model = grid_search.best_estimator_
y_pred_xgb_log = best_xgb_model.predict(X_val_split)

# Invertir la transformación logarítmica para obtener los valores originales
y_pred_xgb = np.expm1(y_pred_xgb_log)

# Calcular el MAE para el modelo optimizado
mae_xgb = mean_absolute_error(y_val_split, y_pred_xgb)
print(f'MAE del modelo optimizado: {mae_xgb}')



Fitting 3 folds for each of 6561 candidates, totalling 19683 fits
Mejores parámetros encontrados: {'alpha': 0.2, 'colsample_bytree': 0.7, 'gamma': 0, 'lambda': 0.1, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1}
MAE del modelo optimizado: 234.1372663187329


In [6]:
# Predecir en el conjunto de prueba
y_pred_test_log = best_xgb_model.predict(X_test_improved)

# Invertir la transformación logarítmica para obtener los valores de precio originales
y_pred_test = np.expm1(y_pred_test_log)

# Crear un DataFrame con los resultados
output_df = test_df[['id']]
output_df['Price_euros'] = y_pred_test

# Guardar el archivo CSV de salida
output_df.to_csv('laptop_price_predictions.csv', index=False)
print('Archivo de predicciones guardado como laptop_price_predictions.csv')


Archivo de predicciones guardado como laptop_price_predictions.csv


C:\Users\mikel\AppData\Local\Temp\ipykernel_21888\2811501838.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['Price_euros'] = y_pred_test
